# Training mit Pytorch Lighting

Das Paket PyTorch lighting dient als zusätzliche Abstraktionsschicht und erspart große Menge an Code, in welchem sonst die Trainingsloops und Logging etc. programmiert werden müsstens. Mit PyTorch lighting können relativ einfach die vorhanden Klassen anpassen und haben einen kompletten Trainingslauf mit Metrikenanzeige und Logging. Diese Funktionen und Klassen können wir dann einzelnd an die einzelnen Modelle anpassen.

## Download der Daten

In [ ]:
import os
import sys
import shutil

# Function to check if running in Google Colab
def in_colab():
    return 'google.colab' in sys.modules

# Define the base path for the dataset
if in_colab():
    !git clone https://github.com/kteppris/RealWaste.git
    %cd RealWaste
    !pip install -r requirements.txt
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Assuming the dataset is in "drive/My Drive/RealWaste" in Google Drive
    dataset_path = '/content/drive/MyDrive/RealWasteData'
else:
    # Path in your local container
    dataset_path = input("Welcher Pfad soll für den Datensatz benutzt werden?")

# Ensure the dataset directory exists (specifically for local container, as Colab will have it in Drive)
if not in_colab():
    os.makedirs(dataset_path, exist_ok=True)  # Create the directory if it doesn't exist

# Clone the repository only if the dataset does not already exist
if not os.listdir(dataset_path):  # Checks if the dataset directory is empty
    !git clone https://github.com/sam-single/realwaste.git {dataset_path}
    # Remove unnecessary files and directories
    readme_path = os.path.join(dataset_path, 'README.md')
    if os.path.exists(readme_path):
        os.remove(readme_path)
    realwaste_dir = os.path.join(dataset_path, 'RealWaste')
    if os.path.isdir(realwaste_dir):
        !mv {realwaste_dir}/* {dataset_path}
        shutil.rmtree(realwaste_dir)  # Remove the now-empty RealWaste directory
    git_dir = os.path.join(dataset_path, '.git')
    if os.path.isdir(git_dir):
        shutil.rmtree(git_dir)  # Remove the .git directory


print("Dataset Path: ", dataset_path)

Welcher Pfad soll für den Datensatz benutzt werden? ~/work/Sonstiges/Module/Machine_Learning/RealWaste/data


fatal: destination path '/home/jovyan/work/Sonstiges/Module/Machine_Learning/RealWaste/data' already exists and is not an empty directory.
Dataset Path:  ~/work/Sonstiges/Module/Machine_Learning/RealWaste/data


## Datamodule

Siehe: https://lightning.ai/docs/pytorch/stable/data/datamodule.html

Im Rahmen des Projekts RealWaste wurde eine spezielle Klasse, `ConfigurableDataModule`, entwickelt, um den Umgang mit Datensätzen für maschinelles Lernen zu vereinfachen und flexibler zu gestalten. Diese Klasse ist eine vereinfachte Anpassung der PyTorch Lightning DataModule-Klasse und ermöglicht es, verschiedene Modelle dynamisch mit unterschiedlichen Datentransformationen zu experimentieren. Ziel ist es, Kollegen, die möglicherweise nicht tief in die Materie des maschinellen Lernens eingetaucht sind, einen leicht zugänglichen Weg zu bieten, um mit Datensätzen zu arbeiten.

#### Kernfunktionalitäten:

- **Flexibilität bei Datentransformationen:** Durch die Übergabe eines `transform`-Objekts beim Initialisieren der Klasse kann der Benutzer spezifische Transformationen definieren, die auf die Bilddaten angewendet werden sollen. Dies ermöglicht eine einfache Anpassung der Vorverarbeitungsschritte für unterschiedliche Modelle oder Experimente.

- **Automatische Aufteilung des Datensatzes:** Im `setup`-Schritt teilt die Klasse den Datensatz automatisch in Trainings- und Validierungsdatensätze auf, basierend auf einem standardmäßigen Verhältnis von 80% Training zu 20% Validierung. Diese Aufteilung hilft dabei, das Modell während des Trainings zu evaluieren und die Generalisierbarkeit zu überprüfen.

- **Einfaches Laden der Daten:** Die `train_dataloader`- und `val_dataloader`-Methoden erstellen DataLoader für die Trainings- und Validierungsdatensätze, welche das Batching der Daten und optional das Durchmischen (nur Trainingsdatensatz) übernehmen. Die Batch-Größe ist dabei ein konfigurierbarer Parameter, der beim Initialisieren der Klasse festgelegt wird. Siehe https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

#### Anwendung:

Um die `ConfigurableDataModule`-Klasse zu nutzen, müssen Benutzer lediglich den Pfad zum Datensatz (`data_dir`), die gewünschte Batch-Größe und das Transformationsschema als Parameter angeben. Nach der Initialisierung kann die Klasse mit PyTorch Lightning Trainern verwendet werden, um das Modelltraining und die Validierung zu vereinfachen.

#### Beispiel:

```python
from torchvision import transforms as T
transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

data_module = ConfigurableDataModule(data_dir='/path/to/dataset', batch_size=32, transform=transform)
```

Mit diesem Ansatz können Projektbeteiligte verschiedene Bildverarbeitungstechniken und Modellarchitekturen schnell evaluieren, ohne tief in die Komplexität der Datenvorbereitung eintauchen zu müssen. Das anpassen der Transforms wird für jedes Modell individuell neu notwendig sein, da wir beispeilsweise die Größe das Bildes für das jeweilige Modell anpassen müssen.

In [ ]:
from pytorch_lightning import LightningDataModule
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import os

class ConfigurableDataModule(LightningDataModule):
    """Class wraper für mit austauschbaren transforms"""
    def __init__(self, data_dir: str, batch_size: int, transform):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transform

    def setup(self, stage=None):
        # Erstellen des Datensatzes als Instanz von ImageFolder
        full_dataset = ImageFolder(root=self.data_dir, transform=self.transform)
        # Setzen der Trainingsset/Validierungsset Größe
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        # Zufälliges aufteilen in Training- und Validierungdatensatz
        self.train_dataset, self.val_dataset = random_split(full_dataset, [train_size, val_size])

    def train_dataloader(self):
        # Setzen des Traindataloader
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=os.cpu_count())

    def val_dataloader(self):
        # Setzen des Validation Dataloader
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=os.cpu_count())

## Lighting Module Basis

Der `BaseWasteClassifier` bildet die Grundlage für eine Reihe von Klassifizierungsmodellen im Projekt RealWaste, das sich mit der Erkennung und Klassifizierung von Abfallbildern beschäftigt. Diese Klasse erbt von PyTorch Lightning's `LightningModule` und bietet ein Grundgerüst, das für alle spezifischen Klassifizierungsmodelle innerhalb des Projekts angepasst werden kann. Ziel ist es, eine konsistente Struktur und gemeinsame Funktionalitäten für das Training, die Validierung und die Metrikbewertung bereitzustellen, um die Entwicklung und Evaluierung von Modellen zu vereinfachen.

#### Kernfunktionalitäten:

- **Modellstruktur:** Die Klasse definiert eine variable `model`, die in abgeleiteten Klassen mit spezifischen Modellarchitekturen (z.B. CNNs für Bildklassifizierung) gefüllt wird. Dies ermöglicht Flexibilität und Wiederverwendbarkeit des Codes.

- **Metriken:** `BaseWasteClassifier` initialisiert standardisierte Metriken für Genauigkeit (Accuracy), Präzision (Precision) und Rückruf (Recall), die für Multiklassen-Klassifizierungsprobleme angepasst sind. Diese Metriken erleichtern die Leistungsbewertung über verschiedene Modelle hinweg.

- **Datenfluss:** Die Methoden `forward`, `training_step`, und `validation_step` definieren den grundlegenden Datenfluss durch das Modell. Dabei ist `forward` für die Vorwärtspropagierung, `training_step` für die Berechnung des Verlusts und der Metriken während des Trainings und `validation_step` für die Auswertung während der Validierung zuständig.

- **Optimierung:** Durch die Methode `configure_optimizers` wird standardmäßig ein SGD-Optimizer definiert. Diese Methode kann in abgeleiteten Klassen überschrieben werden, um spezifische Optimierungsstrategien zu implementieren.

#### Anwendung:

Um ein spezifisches Klassifizierungsmodell zu erstellen, sollten Entwickler eine neue Klasse definieren, die von `BaseWasteClassifier` erbt und die `model`-Variable mit einer geeigneten Modellarchitektur (z.B. ein bestimmtes neuronales Netzwerk) sowie die Methode `forward` zur Definition der Vorwärtspropagierung füllt. Dies ermöglicht eine strukturierte und einheitliche Herangehensweise an die Modellentwicklung.

#### Beispiel:

```python
class SpecificWasteClassifier(BaseWasteClassifier):
    def __init__(self, num_classes):
        super().__init__(num_classes=num_classes)
        self.model = ...  # Definiere hier die spezifische Modellarchitektur

    def forward(self, x):
        # Implementiere die Vorwärtspropagierung
        return self.model(x)
```

Durch diesen Ansatz können Entwickler im RealWaste-Projekt verschiedene Modelle schnell iterieren und evaluieren, wobei sie sich auf das wesentliche Design der Modellarchitektur konzentrieren können, ohne sich jedes Mal um die Implementierungsdetails von Training und Validierung kümmern zu müssen. Es ist wichtig zu betonen, dass `BaseWasteClassifier` speziell für das Projekt RealWaste entwickelt wurde, um eine konsistente und effiziente Entwicklungspraxis zu fördern.

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms.functional import to_pil_image, to_tensor

In [ ]:
import pytorch_lightning as pl
import torch
import torchmetrics
import torchvision
import torchvision.utils as vutils
import random

class BaseWasteClassifier(pl.LightningModule):

    CLASS_NAMES =  ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']

    def __init__(self, num_classes: int):
        super().__init__()
        self.num_classes = num_classes
        # Placeholder for the actual model, to be defined in subclasses
        self.model = None

        # Initialize metrics common to all models
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes, average='macro')
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes, average='weighted')
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes, average='weighted')

    def forward(self, x):
        # Subclasses should implement this method
        raise NotImplementedError("This method should be overridden by subclasses.")

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        predictions = torch.argmax(logits, dim=1)  # Convert logits to predicted class indices

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

        if random.random() < 0.1:  # Log images randomly
            self.log_images_with_labels(x, y, predictions, batch_idx)  # Pass predictions correctly


    def log_images_with_labels(self, images, labels, predictions, batch_idx):
        """Log a batch of images with their actual and predicted labels to TensorBoard."""
        annotated_images = []

        for i in range(images.size(0)):
            img = images[i]
            actual_label = labels[i].item()
            predicted_label = predictions[i].item()

            # Unnormalize the image for visualization
            img = self.unnormalize(img)  # Make sure to call with self if it's an instance method

            # Convert to PIL Image for easy manipulation
            pil_img = to_pil_image(img)
            draw = ImageDraw.Draw(pil_img)
            annotation_text = f'Actual: {self.CLASS_NAMES[actual_label]},\n Predicted: {self.CLASS_NAMES[predicted_label]}'
            draw.text((10, 10), annotation_text, fill="white")

            # Convert back to tensor and add to list
            annotated_img = to_tensor(pil_img)
            annotated_images.append(annotated_img.unsqueeze(0))  # Add batch dimension

        # Stack all annotated images into a single tensor for logging
        annotated_images_tensor = torch.cat(annotated_images, dim=0)
        img_grid = torchvision.utils.make_grid(annotated_images_tensor, nrow=4)

        # Log the grid of annotated images
        self.logger.experiment.add_image(f'Validation Images, Batch {batch_idx}', img_grid, self.current_epoch)


    def unnormalize(self, image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        """Revert normalization of an image tensor."""
        image = image.clone()  # Clone the tensor to avoid in-place operations
        for t, m, s in zip(image, mean, std):
            t.mul_(s).add_(m)  # Multiply by std and add mean
        return image

    def configure_optimizers(self):
        # Subclasses can override this if needed
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        return optimizer


## Trainingsprozess Funktion

Für das Projekt RealWaste wurde eine standardisierte Methode zur Durchführung des Trainingsprozesses von Modellen etabliert. Diese Methode, `train_model`, vereinfacht das Training von Klassifizierungsmodellen durch die Automatisierung gängiger Schritte und die Integration von Logging-Funktionalitäten. Ziel ist es, Entwicklern eine einfache und effiziente Möglichkeit zu bieten, verschiedene Modelle zu trainieren und deren Leistung zu überwachen.

#### Kernfunktionalitäten:

- **Trainingseinrichtung:** Die Funktion nimmt ein Modell und ein Datenmodul als Parameter entgegen und konfiguriert den Trainingsprozess. Das Datenmodul sollte eine Instanz der zuvor beschriebenen `ConfigurableDataModule`-Klasse oder einer ähnlichen Klasse sein, die die Daten für das Training und die Validierung bereitstellt.

- **Logging:** Durch die Verwendung eines `TensorBoardLogger` wird ein Logging-Mechanismus eingerichtet, der es ermöglicht, Metriken und Trainingsfortschritte in TensorBoard zu verfolgen. Dies erleichtert die Analyse des Modellverhaltens über die Zeit und unterstützt die visuelle Auswertung der Modellleistung.

- **Trainingskonfiguration:** Die Funktion erlaubt die Definition der maximalen Anzahl von Epochen durch den Parameter `max_epochs`. Dies gibt an, wie lange das Modell trainiert werden soll. Standardmäßig ist dieser Wert auf 5 gesetzt, kann jedoch für längere Trainingsläufe entsprechend angepasst werden.

- **Flexibilität:** Durch den optionalen Parameter `logger_name` kann der Name des Log-Verzeichnisses angepasst werden, was eine organisierte Speicherung von Trainingslogs für verschiedene Modelle oder Trainingsläufe ermöglicht.

#### Anwendung:

Um ein Modell zu trainieren, erstellen Entwickler zunächst eine Instanz ihres Modells und des Datenmoduls und rufen dann `train_model` mit diesen Instanzen auf. Optional können sie die Anzahl der Trainingsepochen und den Namen des Log-Verzeichnisses anpassen.

#### Beispiel:

```python
model = SpecificWasteClassifier(num_classes=9)  # Ein spezifisches Modell für das Projekt
data_module = ConfigurableDataModule(data_dir='/path/to/dataset', batch_size=32, transform=transform)  # Das Datenmodul

train_model(model, data_module, max_epochs=10, logger_name="waste_classifier_logs")
```

Durch die Verwendung dieser Methode können Entwickler im RealWaste-Projekt den Trainingsprozess ihrer Modelle mit minimalen Aufwand steuern und überwachen, was eine schnelle Iteration und Optimierung der Modelle ermöglicht. Es ist wichtig zu betonen, dass diese Methode speziell für die Bedürfnisse des RealWaste-Projekts entwickelt wurde, um eine konsistente und effektive Trainingspraxis zu fördern.

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

def train_model(model, data_module, log_dir="tb_logs", max_epochs=5, logger_name="model_logs"):
    # Starten das Trainingsprozesses
    logger = TensorBoardLogger(log_dir, name=logger_name)
    trainer = Trainer(max_epochs=max_epochs, logger=logger)
    trainer.fit(model, datamodule=data_module)


# InceptionWasteClassifier

Der `InceptionWasteClassifier` ist eine spezialisierte Implementierung des `BaseWasteClassifier` für das Projekt RealWaste, der das Inception v3-Modell für die Klassifizierung von Abfallbildern nutzt. Diese Klasse erweitert das grundlegende Framework, das durch `BaseWasteClassifier` bereitgestellt wird, um die leistungsstarke Architektur des Inception-Modells zu integrieren und anzupassen. Ziel ist es, eine hochpräzise Bildklassifizierungslösung zu bieten, die auf den spezifischen Anforderungen des RealWaste-Projekts basiert.

#### Kernfunktionalitäten:

- **Integration von Inception v3:** Durch die Nutzung des vortrainierten Inception v3-Modells als Basisarchitektur ermöglicht `InceptionWasteClassifier` eine effektive Feature-Extraktion und Klassifizierung. Das Modell wird mit Standardgewichten initialisiert, was einen guten Ausgangspunkt für das Training bietet.

- **Anpassung an die Zielklassen:** Die finale vollverbundene Schicht (`fc`) des Inception-Modells wird durch eine neue Schicht ersetzt, deren Ausgabegröße der Anzahl der Zielklassen im RealWaste-Projekt entspricht. Dies ist eine gängige Technik, um vortrainierte Modelle für neue Klassifizierungsaufgaben anzupassen.

- **Unterstützung für Hilfsausgaben:** Im Trainingmodus liefert das Modell zusätzlich zu den Hauptlogits Hilfslogits (`aux_logits`), die für die Berechnung des Verlusts verwendet werden können, um die Konvergenz zu verbessern. Diese Dualität wird in der `forward`-Methode gehandhabt, indem je nach Modus entweder beide Outputs oder nur die Hauptlogits zurückgegeben werden.

- **Spezifische Trainingslogik:** Die `training_step`-Methode ist so angepasst, dass sie sowohl mit den Haupt- als auch mit den Hilfslogits arbeiten kann, wobei der Hauptfokus auf den Hauptlogits liegt. Dies unterstützt eine effektive Rückpropagation und Leistungsoptimierung während des Trainings.

#### Anwendung:

Um den `InceptionWasteClassifier` zu verwenden, müssen Entwickler lediglich eine Instanz der Klasse erstellen und das Training mit einem geeigneten Datensatz beginnen. Dank der Erweiterung von `BaseWasteClassifier` erbt der `InceptionWasteClassifier` alle Trainings- und Validierungsmethoden, was eine nahtlose Integration in das Projekt RealWaste ermöglicht.

#### Beispiel:

```python
# Instanziierung des InceptionWasteClassifier
model = InceptionWasteClassifier(num_classes=9)  # Für 9 verschiedene Abfallklassen

# Vorbereitung des Datenmoduls und Start des Trainingsprozesses
data_module = ConfigurableDataModule(data_dir='/path/to/dataset', batch_size=32, transform=transform)
train_model(model, data_module, max_epochs=10, logger_name="inception_waste_classifier_logs")
```

Durch die Verwendung des `InceptionWasteClassifier` können Entwickler im RealWaste-Projekt von den fortschrittlichen Fähigkeiten des Inception v3-Modells profitieren, angepasst an die spezifischen Anforderungen der Abfallbildklassifizierung. Dieser Ansatz fördert nicht nur eine hohe Genauigkeit und Effizienz in der Modellleistung, sondern auch eine schnelle und effektive Entwicklung und Evaluierung von Modellen.

In [ ]:
from torchvision.models import inception_v3

class InceptionWasteClassifier(BaseWasteClassifier):
    def __init__(self, num_classes=9):
        super().__init__(num_classes)
        self.model = inception_v3(aux_logits=True, weights="Inception_V3_Weights.DEFAULT")
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        if self.training:
            outputs = self.model(x)
            return outputs.logits, outputs.aux_logits
        else:
            return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)  # This could be either a tensor or a tuple of tensors
        logits = outputs if isinstance(outputs, torch.Tensor) else outputs[0]  # Select logits
        loss = torch.nn.functional.cross_entropy(logits, y)
        acc = self.accuracy(torch.argmax(logits, dim=1), y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss



## Datenvorbereitung für InceptionWasteClassifier

Um den `InceptionWasteClassifier` im Projekt RealWaste effektiv zu nutzen, ist eine spezifische Datenvorbereitung erforderlich. Diese Vorbereitung umfasst die Anwendung einer Reihe von Transformationen auf die Bilddaten, um sie für das Inception v3-Modell geeignet zu machen. Der folgende Abschnitt beschreibt die notwendigen Schritte und Transformationen, die implementiert werden müssen, um die Bilddaten für das Training und die Validierung des `InceptionWasteClassifier` zu optimieren.

#### Kerntransformationen:

- **Bildgrößenanpassung und Zentrierter Zuschnitt:** Zunächst werden die Bilder auf eine Größe von 342x342 Pixeln skaliert und dann auf eine zentrierte Fläche von 299x299 Pixeln zugeschnitten. Diese Größenanpassung und der Zuschnitt sind spezifisch auf die Anforderungen des Inception v3-Modells abgestimmt, welches Eingaben in dieser Größe erwartet.

- **Konvertierung zu einem Tensor:** Die transformierten Bilder werden in Tensoren umgewandelt, um sie für die Verarbeitung mit PyTorch kompatibel zu machen.

- **Normalisierung:** Die Pixelwerte der Bilder werden normalisiert, indem ein Mittelwert von `[0.485, 0.456, 0.406]` und eine Standardabweichung von `[0.229, 0.224, 0.225]` angewendet werden. Diese Normalisierungswerte sind auf die Verteilung der Daten im Imagenet-Trainingssatz abgestimmt, auf dem das Inception v3-Modell ursprünglich trainiert wurde. Diese Schritte sind entscheidend, um eine konsistente und effektive Modellleistung zu gewährleisten.

#### Anwendung der Transformationen:

Die definierten Transformationen sollten auf alle Bilddaten angewendet werden, die für das Training und die Validierung des `InceptionWasteClassifier` verwendet werden. Dies wird erreicht, indem die Transformationen in das Datenmodul integriert werden, das die Daten für das Modell lädt und vorbereitet.

#### Beispiel:

```python
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Pfad zum Datensatz
data_dir = '/path/to/dataset'

# Erstellung des transformierten Datensatzes
transformed_dataset = ImageFolder(root=data_dir, transform=transforms)

# Laden des Datensatzes als DataLoader für das Training
data_loader = DataLoader(transformed_dataset, batch_size=32, shuffle=True)
```

Durch die Anwendung dieser spezifischen Transformationen können Entwickler sicherstellen, dass die Bilddaten optimal für das Inception v3-Modell vorbereitet sind, wodurch die Genauigkeit und Effizienz des `InceptionWasteClassifier` im Projekt RealWaste maximiert wird. Dieser methodische Ansatz zur Datenvorbereitung ist ein wesentlicher Schritt, um die Leistungsfähigkeit des Modells voll auszuschöpfen und präzise Klassifizierungsergebnisse zu erzielen.

In [ ]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

# Define the transformations as per Inception V3's requirements
transform = transforms.Compose([
    transforms.Resize(342, interpolation=InterpolationMode.BILINEAR),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


model = InceptionWasteClassifier()

## Training the Model

In [ ]:
# Eigene Paket
from utils.colab_helper import in_colab

if in_colab():
    # Load the TensorBoard notebook extension
    %load_ext tensorboard
    %tensorboard --logdir tb_logs
    log_dir = "tb_logs"

else:
    log_dir = input("Bitte den gewünschten Pfad für die Log Datein eingeben:")

Bitte den gewünschten Pfad für die Log Datein eingeben: /home/jovyan/logs


In [ ]:

batch_size = 32
data_module = ConfigurableDataModule(dataset_path, batch_size, transform)

train_model(model, data_module, log_dir=log_dir, max_epochs=1, logger_name="InceptionWaste")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/jovyan/logs/InceptionWaste
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | accuracy  | MulticlassAccuracy  | 0     
1 | precision | MulticlassPrecision | 0     
2 | recall    | MulticlassRecall    | 0     
3 | model     | Inception3          | 25.1 M
--------------------------------------------------
25.1 M    Trainable params
0         Non-trainable params
25.1 M    Total params
100.523   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


# Vision Transformer Training

In [ ]:
# Eigene Paket
from utils.colab_helper import in_colab

if in_colab():
    # Load the TensorBoard
    %tensorboard --logdir tb_logs

In [ ]:
from torchvision.models import vit_b_16  # Ensure this matches your torchvision version and model choice

class ViTWasteClassifier(BaseWasteClassifier):
    def __init__(self, num_classes: int):
        super().__init__(num_classes)
        self.model = vit_b_16(weights="ViT_B_16_Weights.DEFAULT")
        # Access the 'head' layer within 'heads' to replace it for the number of target classes
        in_features = self.model.heads[0].in_features  # Assuming 'head' is the first layer in 'heads'
        self.model.heads[0] = torch.nn.Linear(in_features, num_classes)

    def forward(self, x):
        # For ViT, simply return the model's output
        return self.model(x)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
data_module = ConfigurableDataModule(data_dir=dataset_path, batch_size=32, transform=transform)
vit_model = ViTWasteClassifier(num_classes=9)

In [ ]:
train_model(model=vit_model, data_module=data_module, log_dir=log_dir, max_epochs=5, logger_name="ViT")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-02-12 01:47:09.321785: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 01:47:09.421343: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory fo

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
